In [6]:
import networkx as nx
import numpy as np
import json 

# --- OpenQAOA Imports ---
from openqaoa import QAOA, create_device
from openqaoa.problems import MaximumCut
import qiskit
from qiskit import qasm3

In [10]:
NUM_QUBITS = 6
GRAPH_SEED = 42
DEPTH = 3 # Number of QAOA layers
MIXER = 'xy' # or 'x'
PARAM_TYPE = 'standard' # or 'extended', 'fourier', etc.
OPTIMIZER_METHOD = 'COBYLA'
OPTIMIZER_MAXITER = 100
OPTIMIZER_TOL = 1e-2
OUTPUT_QASM_FILE = 'parameterized_circuit.qasm3'
OUTPUT_SETTINGS_FILE = 'qaoa_settings.json'

In [11]:
g = nx.generators.fast_gnp_random_graph(n=NUM_QUBITS, p=0.6, seed=GRAPH_SEED)

# 2. Create QUBO Problem
graphProblem = MaximumCut(g)
qubo_problem = graphProblem.qubo

# 3. Create QAOA object and set parameters
q = QAOA()

intendedDevice = create_device(location="ibmq", name="ibm_brisbane")
q.set_device(intendedDevice)

q.set_circuit_properties(
    p=DEPTH,
    param_type=PARAM_TYPE,
    init_type="ramp", # init_type mainly affects OpenQAOA's optimizer start
    mixer_hamiltonian=MIXER
)

q.set_backend_properties(init_hadamard=True)

q.set_classical_optimizer(method=OPTIMIZER_METHOD, maxiter=OPTIMIZER_MAXITER, tol=OPTIMIZER_TOL) #idk if i really need to do this here but it might be required by compile()

q.compile(qubo_problem)

/Users/kv18799/Github/QAOAstudygit/openQAOAvenv/lib/python3.10/site-packages/networkx/readwrite/json_graph/node_link.py:142: FutureWarning: 
The default value will be `edges="edges" in NetworkX 3.6.

To make this warning go away, explicitly set the edges kwarg, e.g.:

  nx.node_link_data(G, edges="links") to preserve current behavior, or
  nx.node_link_data(G, edges="edges") for forward compatibility.
  warnings.warn(


In [29]:
variational_params = q.optimizer.variational_params
#q.backend.qaoa_circuit(variational_params).draw()

In [7]:
parameterized_circuit = q.backend.parametric_circuit #idk why this doesnt work anymore it ran earlier
# Get parameters in the order defined by the circuit/OpenQAOA
# Convert to list of strings (parameter names) for saving
parameter_names = [p.name for p in parameterized_circuit.parameters]

print(f"Exporting circuit to {OUTPUT_QASM_FILE}...")
# Use qiskit.qasm3 exporter available in the environment
qasm3_string = qiskit.qasm3.dumps(parameterized_circuit)
with open(OUTPUT_QASM_FILE, 'w') as f:
    f.write(qasm3_string)
print("Circuit export successful.")

Exporting circuit to parameterized_circuit.qasm3...
Circuit export successful.


In [25]:
settings_data = {
    "p_layers": DEPTH,
    "num_qubits": NUM_QUBITS,
    "mixer": MIXER,
    "param_type": PARAM_TYPE,
    "optimizer_method": OPTIMIZER_METHOD,
    "optimizer_maxiter": OPTIMIZER_MAXITER,
    "optimizer_tol": OPTIMIZER_TOL,
    "parameter_names_ordered": parameter_names,
    "graph_nodes": list(g.nodes),
    "graph_edges": list(g.edges),
    "graph_edge_weights": {f"{u}-{v}": g.get_edge_data(u, v) for u, v in g.edges} # Store weights if needed
}

print(f"Saving settings to {OUTPUT_SETTINGS_FILE}...")

with open(OUTPUT_SETTINGS_FILE, 'w') as f:
    json.dump(settings_data, f, indent=4)

# This print statement is correctly outside the block
print("Settings saved successfully.")

print("\nCircuit and settings export complete.")

Saving settings to qaoa_settings.json...
Settings saved successfully.

Circuit and settings export complete.
